# Imports

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import csv
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
from sklearn.metrics import mean_squared_error 
from sklearn.preprocessing import LabelEncoder

# Importing, Splitting and Normalisng Dataset

In [13]:
modelEncoder = LabelEncoder()
transmissionEncoder = LabelEncoder()
fuelTypeEncoder = LabelEncoder()
scaler = MinMaxScaler()

def dataset(brand):
    file = pd.read_csv(brand, quotechar='"', skipinitialspace=True)

    for i in ['year']:
        q75,q25 = np.percentile(file.loc[:,i],[75,25])
        IQR = q75-q25
    
        max = q75+(1.5*IQR)
        min = q25-(1.5*IQR)
    
        file.loc[file[i] < min, i] = np.nan
        file.loc[file[i] > max, i] = np.nan

    file = file.dropna(axis = 0)


    modelEncoder.fit(file["model"])
    file["model"] = modelEncoder.transform(file["model"])
    
    transmissionEncoder.fit(file["transmission"])
    file["transmission"] = transmissionEncoder.transform(file["transmission"])
    
    fuelTypeEncoder.fit(file["fuelType"])
    file["fuelType"] = fuelTypeEncoder.transform(file["fuelType"])

    file = file.head(10000) # Limits dataset size

    X = file.drop(columns = ['price'])
    Y = file.price

#     print(file)
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 601)
    
    scaler.fit(X_train)

    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    print(list(fuelTypeEncoder.classes_))

    return  X_train, X_test, Y_train, Y_test, file

In [15]:
X_train, X_test, Y_train, Y_test, file = dataset("../UKUsedCarDataSet/audi.csv") # Use Audi dataset as default for KNN analysis

['Diesel', 'Hybrid', 'Petrol']


# KNN using Scikit-Learn

from sklearn.neighbors import KNeighborsRegressor as KNN_Reg
knn = KNN_Reg(n_neighbors = 4)
knn.fit(X_train, Y_train)
print(knn.score(X_test, Y_test))

results = np.empty(99)
for k in range(1, 100):
    knn = KNN_Reg(n_neighbors = k)
    knn.fit(X_train, Y_train)
    results[k - 1] = (knn.score(X_test, Y_test))

plt.plot(np.arange(99) + 1, results)
plt.xlabel('K Neighbors')
plt.ylabel('R^2')

GRIDSEARCH

In [1]:
# from sklearn.model_selection import GridSearchCV
# k_range = list(range(1, 100))
# param_grid = dict(n_neighbors = k_range)
  
# # defining parameter range
# grid = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy', return_train_score=False, verbose=1)
  
# # fitting the model for grid search
# grid_search = grid.fit(X_train, Y_train)

print(grid_search.best_params_)

# Euclidean Distance

In [22]:
def eucDistance(variable1, variable2):
    distance = 0
    for i in range(len(variable2)):
        distance += (variable1[i] - variable2[i])**2
    return np.sqrt(distance)

# KNN

In [23]:
def kNN(train, testRow, yTrain, num):
    distance = list() #Stores distance of each point
    for i in range(len(train)): #REMOVED -1 from here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        dist = eucDistance(train[i], testRow) #sends points to work out distance
        distance.append((train[i], dist, yTrain.iloc[i])) #CHNAGEEE
    distance = sort(distance) #need to create a sort method to order list
#     print("DISTANCE: ", distance)
    kNeighbours = list() #list to store K amount of neighbour results
    for i in range(num):
        kNeighbours.append((testRow, distance[i][1], distance[i][2]))
    return kNeighbours 

# Sort Method

In [24]:
def sort(dist):
    for i in range(0, len(dist)):
        for j in range(0, len(dist) - i - 1):
            if (dist[j][1] > dist[j + 1][1]):
                temp = dist[j]
                dist[j] = dist[j + 1]
                dist[j + 1] = temp
    return dist

# Predict Method

In [25]:
from statistics import mean 

def predict(train, test, yTrain, num_neighbors):
    predictions = list()
    for i in range(len(test)):
        neighbour = kNN(train, test[i], yTrain, num_neighbors) #CHANGE
        labels = [] #Stores yTrain for each test variable
        for i in range (len(neighbour)):
            t = labels.append(neighbour[i][2]) #Appends yTrain
        predictions.append(mean(labels))
    return predictions

# RMSE

In [27]:
def rmse(test, pred):
    MSE = np.square(np.subtract(test, pred)).mean()
    return sqrt(MSE)

# Evaluation Method

def eva(train, test, yTrain, yTest):
    rmseValues = []
    predictions = []
    for k in range(1, 101): #Testing up to 100 neearest neighbours
        y_pred = predict(train, test, yTrain, k)
        error = rmse(yTest, y_pred) 
        rmseValues.append(error)
        print('RMSE value for k =' , k , 'is:', error)

    plt.plot(np.arange(100) + 1, rmseValues)
    plt.xlabel('K Neighbors')
    plt.ylabel('RMSE')
    
    return

eva(X_train, X_test, Y_train, Y_test)

# 1NN

In [13]:
def NN(train, test, yTrain, yTest):    
    y_pred = predict(train, test, yTrain, 4)
    error = rmse(yTest, y_pred) 
    print('RMSE value for k =' , 4 , 'is:', round(error, 2))
    
    mean= np.mean(file["price"])

    print("R VALUE:", 1 - (error/mean))
    
    ### Below is attempt to put data on bar chart
#     YTest = pd.DataFrame(yTest)
# #     y_pred = pd.DataFrame(y_pred)
# #     y_pred = pd.DataFrame({'Price_prediction': y_pred.flatten()})
# #     Comparison = yTest.join(y_pred)
#     plt.bar(yTest, y_pred)
# #     plt.grid(linestyle = '-', linewidth = '0.6', color = 'green', which = 'both')
#     plt.show()
    return 
NN(X_train, X_test, Y_train, Y_test)

RMSE value for k = 4 is: 3447.6766234668817

Original value: 18498 vs the predicted value: 17370
The difference is: 1128

Original value: 22995 vs the predicted value: 22546
The difference is: 449

Original value: 12495 vs the predicted value: 15170
The difference is: -2675

Original value: 14500 vs the predicted value: 14370
The difference is: 130

Original value: 24995 vs the predicted value: 25618
The difference is: -623

Original value: 10200 vs the predicted value: 11275
The difference is: -1075

Original value: 21995 vs the predicted value: 19542
The difference is: 2453

Original value: 16800 vs the predicted value: 17785
The difference is: -985

Original value: 12798 vs the predicted value: 16346
The difference is: -3548

Original value: 20498 vs the predicted value: 23744
The difference is: -3246

Original value: 11498 vs the predicted value: 14368
The difference is: -2870

Original value: 48136 vs the predicted value: 51347
The difference is: -3211

Original value: 32485 vs th

# User Input Predictions

inputPred = []
entries = []

def userInput():
    chooseBrand = input("Choose your car brand: Audi, BMW, Ford, Hyundai, Mercedes, Skoda, Toyota, Vauxhall or Volkswagen \n")
    
    if chooseBrand == "Audi":
        return "../UKUsedCarDataSet/audi.csv"
    elif chooseBrand == "BMW":
        return "../UKUsedCarDataSet/bmw.csv"
    elif chooseBrand == "Ford":
        return "../UKUsedCarDataSet/ford.csv"
    elif chooseBrand == "Hyundai":
        return "../UKUsedCarDataSet/hyundi.csv"
    elif chooseBrand == "Mercedes":
        return "../UKUsedCarDataSet/merc.csv"
    elif chooseBrand == "Skoda":
        return "../UKUsedCarDataSet/skoda.csv"
    elif chooseBrand == "Toyota":
        return "../UKUsedCarDataSet/toyota.csv"
    elif chooseBrand == "Vauxhall":
        return "../UKUsedCarDataSet/vauxhall.csv"
    elif chooseBrand == "Volkswagen":
        return "../UKUsedCarDataSet/vw.csv"
    else:
        print("Invalid Car Brand")
        userInput()
    return 
  
X_train, X_test, Y_train, Y_test = dataset(userInput())

print("\n List of models:")
print(list(modelEncoder.classes_))

inputPred.append((modelEncoder.transform([input("\nWhat Model is your car? ")]))[0])
inputPred.append(int(input("What year is your car? ")))
inputPred.append((transmissionEncoder.transform([input("What transmission is your car? ")]))[0])
inputPred.append(int(input("How much mileage does your car have? ")))
inputPred.append((fuelTypeEncoder.transform([input("What's your car fuel type? ")]))[0])
inputPred.append(int(input("How much is your cars tax? ")))
inputPred.append(float(input("What's MPG of your car? ")))
inputPred.append(float(input("What the engine size of your car? ")))
entries.append(inputPred)
inputPred = scaler.transform([inputPred])

import time
print("\n ***Predicting***")
start = time.time()
y_pred = predict(X_train, inputPred, Y_train, 4)
# {0:.2f}'.format()
print("\n Predicted price for your car is: £", round(y_pred[0], 2))

print("\n ***Predicted in", time.time() - start,"seconds***")

# Audi,RS6,2016,Semi-Auto,49050,Petrol,325,29.4,4.0    Price = £44,985 Pred:44,717
# BMW,5 Series,2019,Semi-Auto,4405,Petrol,145,48.7,2.0     Price = £26,000
# Skoda,Octavia,2019,Manual,14449,Petrol,150,55.4,1.5      Price = £13,490 Pred: £14,170

testy = 44985
pred = 44717
print(rmse(testy, pred))

# Scikit-Learn User Input Predictions

knn.predict(inputPred)